In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob

from tqdm import tqdm
from numpy.linalg import inv
from scipy.interpolate import InterpolatedUnivariateSpline

In [2]:
pgf_with_latex = {
    "pgf.texsystem": "pdflatex",
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": [],
    "font.sans-serif": [],
    "font.monospace": [],
    "axes.labelsize": 20,
    "font.size": 20,
    "legend.fontsize": 17,
    "xtick.labelsize": 17,
    "ytick.labelsize": 17,
    "pgf.preamble": [
        r"\usepackage[utf8x]{inputenc}",
        r"\usepackage[T1]{fontenc}",
        ]
    }
mpl.rcParams.update(pgf_with_latex)


def savefig(filename):
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')

def savefig_pdf(filename):
    plt.savefig('{}.pdf'.format(filename), bbox_inches='tight')

/home/zdenek/anaconda3/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


<img src="boundaries.png">

In [3]:
###########################################################################
######################## Crestani+2021 boundaries #########################
###########################################################################
ca_slice = np.array([3910.00, 3925.00, 3940.00, 3955.00, 3923.67, 3943.67])
hd_slice = np.array([4010.00, 4060.00, 4145.00, 4195.00, 4091.74, 4111.74])
hg_slice = np.array([4230.00, 4280.00, 4400.00, 4450.00, 4330.47, 4350.47])
hb_slice = np.array([4750.00, 4800.00, 4920.00, 4970.00, 4851.33, 4871.33])
###########################################################################

lines_centers = np.array([3933.66, 4101.75, 4340.47, 4861.34])



###########################################################################################################
###########################################################################################################

def get_shift_wave(wave, flux, eflux, deg):
    """ Centering line """
    x_fit = np.linspace(min(wave), max(wave), 1000)
    
    b_fit = np.polyfit(wave, flux, 
                       w=eflux**-2, deg=4)
    
    wave_off = x_fit[np.argmin(np.polyval(b_fit, x_fit))]
    
    
    slicer_shift = np.where(abs(lines_centers - wave_off) == min(abs(lines_centers - wave_off)))
    
    wave_shift = lines_centers[slicer_shift] - wave_off
    
    return wave_shift


###########################################################################################################
###########################################################################################################

def get_it_done(wave, flux, eflux, flux_orig, slicing, 
                line="", plott=False, name_fig=""):
    
    """Implementation of the deltaS method.

    Parameters
    ----------
    wave : 'array_like, shape (N, )'
        Wavelenght in angstroms 
    flux : 'array_like, shape (N, )'
        Flux in any given unit
    eflux : 'array_like, shape (N, )'
        Uncertainties on flux
    flux_orig : 'array_like, shape (N, )'
        Original flux in any given unit
    slicing : 'array_like, shape (N, )'
        List of wavelenght regions
    line : 'str'
        Name of the spectral line
    name_fig : 'str'
        Name of the figure
        
    Returns:
    ------- 
    area_under_line : 'float'
        Integrated area under spline
    snr : 'float'
        Signal-to-noise ratio
    """     
    
    slice_for_center = np.where((wave > slicing[4]) & (wave < slicing[5]))
    
     
    wave_shift = get_shift_wave(wave[slice_for_center], 
                                flux_orig[slice_for_center], 
                                eflux[slice_for_center], deg=4)   
    wave += wave_shift[0]
    
    
    con_slicer = np.where(((wave > slicing[0]) & (wave < slicing[1])) 
                          | ((wave > slicing[2]) & (wave < slicing[3])))

    
    blue, red = slicing[4], slicing[5]
    p = np.polyfit(wave[con_slicer], flux[con_slicer], deg=1, w=eflux[con_slicer]**-2)

    
    conti_flux = 1 + (flux - np.polyval(p, wave)) / np.polyval(p, wave)
    conti_err = (1 + (flux + eflux - np.polyval(p, wave)) / np.polyval(p, wave)) - conti_flux

    flux_flip = -1*conti_flux + 1
    
    snr = 1./np.std(flux_flip[con_slicer])

    flux_flip[flux_flip < 0] = 0.
    f = InterpolatedUnivariateSpline(wave, 
                                     flux_flip, 
                                     w=conti_err**-2, k=1)

    if plott: plot_image(wave, flux, conti_flux, p, slicing, f, conti_err, line, name_fig)
    
    area_under_line = f.integral(blue, red)
    
    return (area_under_line, snr)

###########################################################################################################
###########################################################################################################

def plot_image(wave, flux, conti_flux, p, slicing, f, conti_err, line, name_fig):
    """Just figure, don't overthink it...
    """ 
    
       
    fig, ax = plt.subplots(figsize=(7.,10.), nrows=2, ncols=1) 

    ax[0].plot(wave, flux, c="k", label="Spectrum")
    ax[0].plot(wave, np.polyval(p, wave), c="C3", ls=":", label="Linear fit")
    ax[0].axvspan(slicing[0], slicing[1], facecolor="C2", alpha=0.15, 
                  hatch='\/', edgecolor="C2", label="Continuum region")
    ax[0].axvspan(slicing[2], slicing[3], facecolor="C2", alpha=0.15, hatch='\/', edgecolor="C2")
    
    ax[0].annotate(r'%s' %(line), 
             xytext=(np.percentile(slicing, 46), np.average(flux)+np.std(flux)),
             xycoords='data',
             textcoords='data',
             xy=(np.percentile(slicing, 46), np.average(flux)+np.std(flux)),
             fontsize=22, color="k") 
    
    ax[0].set_ylabel(r"Flux\,[10$^{-17}$ erg/cm$^2$/s/$\mathrm{\AA}$]")
    ax[0].minorticks_on()
    plt.setp(ax[0].get_xticklabels(), visible=False)
    
    ######################################################################
    
    ax[1].plot(wave, conti_flux, c="k")
    ax[1].scatter(wave, conti_flux, c="k", marker=".", zorder=-10) #, rasterized=True
    ax[1].errorbar(wave, conti_flux, yerr=conti_err, c="k", marker=".", zorder=-10, alpha=0.3)
    ax[1].axhline(1., c="C3", ls="-", zorder=-10, label="Continuum")

    xs = np.linspace(slicing[4], slicing[5], 1000)
    ax[1].plot(xs, f(xs)*(-1)+1, 'C0', lw=3, alpha=0.7)
    ax[1].fill_between(xs, 1, f(xs)*(-1)+1, color="C0", alpha=0.2, label="Pseudo EW")#, rasterized=True
    
    ax[1].set_xlabel(r"Wavelength\,[$\mathrm{\AA}$]")
    ax[1].set_ylabel(r"Normalized flux")
    ax[1].minorticks_on()

    ######################################################################
    
    fig.legend(bbox_to_anchor=(0.85, 1.12), ncol=2)
    fig.tight_layout()
    
    savefig_pdf("figures/"+name_fig)
    plt.close()


In [6]:
names = glob.glob("ForDeltaS/*.txt")

def open_file(nam):
    data = np.genfromtxt(nam, dtype=float, skip_header=0)
    wave = data[:,0]
    flux = data[:,1]
    eflux = data[:,2]
    
    slice_bad = np.where(eflux/flux < 10.)
    
    return wave[slice_bad], flux[slice_bad], eflux[slice_bad]

In [7]:
rep_names = [names[i].replace("ForDeltaS/", "") 
             for i in range(len(names))]

soubor = open("CheckPEW2.dat", "w")
# print("#ID Ca eCa snr_ca Hd eHd snr_hd Hg eHg snr_hg Hb eHb snr_hb\n")
soubor.write("#ID Ca eCa snr_ca Hd eHd snr_hd Hg eHg snr_hg Hb eHb snr_hb\n")


for j in tqdm(range(len(rep_names))):
    wave, flux, eflux = open_file(names[j])
    
    #### Selecting portions of the spectra to get important sections ####
    ca_whole = np.where((wave > min(ca_slice)) & (wave < max(ca_slice)))#
    hd_whole = np.where((wave > min(hd_slice)) & (wave < max(hd_slice)))#
    hg_whole = np.where((wave > min(hg_slice)) & (wave < max(hg_slice)))#
    hb_whole = np.where((wave > min(hb_slice)) & (wave < max(hb_slice)))#
    #####################################################################
    
    #print(names[j])
    num_it = 500
    ew_ca, ew_hd, ew_hg, ew_hb = np.zeros(num_it), np.zeros(num_it), np.zeros(num_it), np.zeros(num_it)
    snr_ca, snr_hd, snr_hg, snr_hb = np.zeros(num_it), np.zeros(num_it), np.zeros(num_it), np.zeros(num_it)
    
    for i in range(num_it):

        flux_r = np.random.normal(flux, eflux)

        ew_ca[i], snr_ca[i] = get_it_done(wave[ca_whole], flux_r[ca_whole],
                                               eflux[ca_whole], flux[ca_whole], ca_slice)  
        
        ew_hd[i], snr_hd[i] = get_it_done(wave[hd_whole], flux_r[hd_whole],
                                               eflux[hd_whole], flux[hd_whole], hd_slice)
        
        ew_hg[i], snr_hg[i] = get_it_done(wave[hg_whole], flux_r[hg_whole],
                                               eflux[hg_whole], flux[hg_whole], hg_slice)
        
        ew_hb[i], snr_hb[i] = get_it_done(wave[hb_whole], flux_r[hb_whole],
                                               eflux[hb_whole], flux[hb_whole], hb_slice)
    
    rad = "%s %.3f %.3f %.3f %.3f %.3f %.3f %.3f %.3f %.3f %.3f %.3f %.3f\n" %(rep_names[j], 
                                                                               np.nanmean(ew_ca), 
                                                                               np.nanstd(ew_ca), 
                                                                               np.median(snr_ca), 
                                                                               np.nanmean(ew_hd), 
                                                                               np.nanstd(ew_hd), 
                                                                               np.median(snr_hd), 
                                                                               np.nanmean(ew_hg), 
                                                                               np.nanstd(ew_hg), 
                                                                               np.median(snr_hg), 
                                                                               np.nanmean(ew_hb), 
                                                                               np.nanstd(ew_hb), 
                                                                               np.median(snr_hb))

    soubor.write(rad)
soubor.close()

100%|██████████| 13/13 [00:15<00:00,  1.23s/it]
